In [1]:
import numpy as np
import healpy as hp
from astropy.io import ascii
from astropy.io import fits
import matplotlib.pyplot as plt
from astropy.table import Table
from astropy.table import Table, Column
%matplotlib inline

In [2]:
print 'Reading files'
w = Table.read('wen.csv') 
r = Table.read('redmapper.fits') 
k = Table.read('koester2007.vot', format='votable')
wz = Table.read('wazp.fits')
print 'Wen:',len(w)
print 'Rykoff:',len(r)
print 'Koester:', len(k)
print 'Wzap:', len(wz)

Reading files
Wen: 132684
Rykoff: 26111
Koester: 13823
Wzap: 18089


In [3]:
NSIDE = 32

In [4]:
# Phi e theta do Wen
phi_w = np.deg2rad(w['RAJ2000'])
theta_w = np.deg2rad(90.-w['DEJ2000'])

In [5]:
#Phi e theta do Rykoff
phi_r = np.deg2rad(r['RA'])
theta_r = np.deg2rad(90.-r['DEC'])

In [6]:
#Phi e theta do Koester
phi_k = np.deg2rad(k['RAJ2000'])
theta_k = np.deg2rad(90.-k['DEJ2000'])

In [7]:
#Phi e theta do Wzap
phi_wz = np.deg2rad(wz['ra_init'])
theta_wz = np.deg2rad(90.-wz['dec_init'])

In [8]:
# Pixels do Wen
wpix = hp.ang2pix(NSIDE, theta_w, phi_w)

In [9]:
# Pixels do Rykoff
rpix = hp.ang2pix(NSIDE, theta_r, phi_r)

In [10]:
#Pixels do Koester
kpix = hp.ang2pix(NSIDE, theta_k, phi_k)

In [11]:
#Pixels do Wzap
wzpix = hp.ang2pix(NSIDE, theta_wz, phi_wz)

In [12]:
#Retira os numeros de pixels repetidos
uwpix = np.unique(wpix)
urpix = np.unique(rpix)
ukpix = np.unique(kpix)
uwzpix = np.unique(wzpix)
print len(wpix), len(uwpix)
print len(rpix), len(urpix)
print len(kpix), len(ukpix)
print len(wzpix), len(uwzpix)

132684 4900
26111 3288
13823 2379
18089 89


In [16]:
# Match de wpix com rpix
lista = []
rp = range(0,len(rpix))
wp = range(0,len(wpix))
for wp in wpix: 
    for rp in rpix:
        if wp==rp:
            lista.append(wp)

KeyboardInterrupt: 

In [13]:
print len(lista)

KeyboardInterrupt: 

In [ ]:
#Match da lista com o kpix
lista_m = []
kp = range(0,len(kpix))
lp = range(0,len(lista))
for kp in kpix:
    for lp in lista:
        if lp==kp:
            lista_m.append(kp)

In [ ]:
print len(lista_m)

In [ ]:
#Match da lista_m com o wzpix
lista_match = []
wzp = range(0,len(wzpix))
lp = range(0,len(lista))
for wzp in wzpix:
    for lp in lista:
        if lp==wzp:
            lista_match.append(wzp)

In [ ]:
print len(lista_match)

In [ ]:
#sysmap = np.zeros(hp.nside2npix(NSIDE),dtype=np.float32) +hp.UNSEEN
mmap = np.zeros(hp.nside2npix(NSIDE), dtype=np.int8) + hp.UNSEEN
wmap = np.zeros(hp.nside2npix(NSIDE), dtype=np.int8) + hp.UNSEEN
rmap = np.zeros(hp.nside2npix(NSIDE), dtype=np.int8) + hp.UNSEEN
kmap = np.zeros(hp.nside2npix(NSIDE), dtype=np.int8) + hp.UNSEEN
wzmap = np.zeros(hp.nside2npix(NSIDE), dtype=np.int8) + hp.UNSEEN

In [ ]:
wmap[wpix] = 1
rmap[rpix] = 1
kmap[kpix] = 1 
mmap[lista_m] = 1
wzmap[lista_match] = 1

In [ ]:
hp.mollview(wmap, fig=1, cmap='jet')
hp.mollview(rmap, fig=2, cmap='jet')
hp.mollview(kmap, fig=3, cmap='jet')
hp.mollview(mmap, fig=4, cmap='jet')
hp.mollview(wzmap, fig=5, cmap='jet')

In [ ]:
# wen
mask_w = np.zeros(len(wpix), int)
goodies = np.array(lista_match)
for i,wp in enumerate(wpix):
    for good in goodies:
        if wp == good:
            mask_w[i] = 1

In [ ]:
print mask_w
print len(mask_w), sum(mask_w)

In [ ]:
# Vai adicinar a nova lista apenas os indices que contém 1 (goodies matches)
match_iw = []
for i in range(0,len(mask_w)):
    if mask_w[i]!=0:
        match_iw.append(i)
print len(match_iw)

In [ ]:
# Aplicando os indices no wen
new_w = w[match_iw]

In [ ]:
# rykoff
mask_r = np.zeros(len(rpix), int)
goodies = np.array(lista_match)
for i,rp in enumerate(rpix):
    for good in goodies:
        if rp == good:
            mask_r[i] = 1

In [ ]:
print mask_r
print len(mask_r), sum(mask_r)

In [ ]:
# Vai adicinar a nova lista apenas os indices que contém 1 (goodies matches)
match_ir = []
for i in range(0,len(mask_r)):
    if mask_r[i]!=0:
        match_ir.append(i)
print len(match_ir)

In [ ]:
# Aplicando os indices no rykoff
new_r = r[match_ir]

In [ ]:
# koester
mask_k = np.zeros(len(kpix), int)
goodies = np.array(lista_match)
for i,kp in enumerate(kpix):
    for good in goodies:
        if kp == good:
            mask_k[i] = 1

In [ ]:
print mask_k
print len(mask_k), sum(mask_k)

In [ ]:
# Vai adicinar a nova lista apenas os indices que contém 1 (goodies matches)
match_ik = []
for i in range(0,len(mask_k)):
    if mask_k[i]!=0:
        match_ik.append(i)
print len(match_ik)

In [ ]:
# Aplicando os indices no koester
new_k = k[match_ik]

In [ ]:
# wazp
mask_wz = np.zeros(len(wzpix), int)
goodies = np.array(lista_match)
for i,wzp in enumerate(wzpix):
    for good in goodies:
        if wzp == good:
            mask_k[i] = 1

In [ ]:
print mask_wz
print len(mask_wz), sum(mask_wz)

In [ ]:
# Vai adicinar a nova lista apenas os indices que contém 1 (goodies matches)
match_iwz = []
for i in range(0,len(mask_wz)):
    if mask_wz[i]!=0:
        match_iwz.append(i)
print len(match_iwz)

In [ ]:
# Aplicando os indices no wazp
new_wz = k[match_iwz]

In [ ]:
new_w.write('new_w.fits', format='fits')

In [ ]:
new_r.write('new_r.fits', format='fits')

In [ ]:
new_k.write('new_k.fits', format='fits')

In [ ]:
new_wz.write('new_wz.fits', format='fits')